## Business Problem
 
 The Objective of this Capstone Project is to located the best Location to open new Bakery Outlet in City New Delhi, India. Using Data Science and Machine Learning Technique, this Project aim to answer this Business problem. With this help of Machine learning we are able to sorted out the location which is well connected to Airport/Bus Station and other public places. The Solution recommend best neighbourhood to open the Bakery.


## Data Description


The data set that I have used for solving the problem is:
- A complete list of neighborhoods in New Delhi, India. Source of the data is
Wikipedia.org (https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi)
- Geographical coordinates (latitude and longitude) of those neighborhoods. Source of
the data will be FourSquare.
- FourSquare provided Venue data which is related to Bakery. Machine Learning
Technique called Clustering will be used for solving the problem.
- Clustering using K-Means clustering algorithm


## Methodology

### Importing

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

import geocoder # to get coordinates
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


print("Libraries imported.")

Libraries imported.


### Exploring  New Delhi
Neighbourhoods of Delhi.
We begin to start collecting and refining the data needed for the our business solution to work.

#### Data Prepare and Preprocessing

In [2]:
url_delhi= requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi").text

In [3]:
# Prepare a DataFrame using the wiki-data
soup = BeautifulSoup(url_delhi, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
df_delhi = pd.DataFrame({"Neighborhood": neighborhoodList[1:]})

print("Total no of neighborhoods:",df_delhi.shape[0])
df_delhi.head()

Total no of neighborhoods: 152


,Neighborhood
0,Ashok Nagar (Delhi)
1,Ashok Vihar
2,Ashram Chowk
3,Babarpur
4,"Badarpur, Delhi"


###  Feature Selection & Engineering



In [4]:
df_delhi.shape



(152, 1)

In [5]:
df_delhi.dtypes

Neighborhood    object
dtype: object

#### Geolocation of New Delhi Neighborhood 

In [6]:
def get_lat_long(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    #print(lat_lng_coords)
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [7]:
df_delhi['Latitude'], df_delhi['Longitude'] = zip(*df_delhi['Neighborhood'].apply(get_lat_long))

In [8]:
df_delhi.head()

,Neighborhood,Latitude,Longitude
0,Ashok Nagar (Delhi),28.692230,77.301240
1,Ashok Vihar,28.690370,77.176090
2,Ashram Chowk,28.710598,77.326965
3,Babarpur,28.507380,77.303460
4,"Badarpur, Delhi",28.507380,77.303460


#### Map Visualization - Neighborhood of Delhi

In [9]:
address = 'Delhi, India'

geolocator = Nominatim(user_agent="delhi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Delhi are {}, {}.'.format(latitude, longitude))

The coordinates of Delhi are 28.6273928, 77.1716954.


The approach taken here is to explore the city, plot the map to show the neighbourhoods being considered and then build our model by clustering all of the similar neighbourhoods together and finally plot the new map with the clustered neighbourhoods. We draw insights and then compare and discuss our findings.

In [10]:
# create map of Delhi using latitude and longitude values
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_delhi['Latitude'], df_delhi['Longitude'], df_delhi['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_delhi)  
    
print("Neighborhood of Delhi")
map_delhi

Neighborhood of Delhi


In [11]:
# save the map as HTML file
map_delhi.save('map_delhi.html')

###  FourSquare APIs

In [12]:
CLIENT_ID = 'K4SW4CPISLQT2WQ420U425WJW4OLGAWXO0FCDZHJQK5RAE1B' # your Foursquare ID
CLIENT_SECRET = 'TLMRBWWY2FPC1BAG0NAE4OPUCEO522PBRTEXAWGJ3ZXFP2LD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K4SW4CPISLQT2WQ420U425WJW4OLGAWXO0FCDZHJQK5RAE1B
CLIENT_SECRET:TLMRBWWY2FPC1BAG0NAE4OPUCEO522PBRTEXAWGJ3ZXFP2LD


In [13]:
df_delhi.loc[0, 'Neighborhood']

'Ashok Nagar (Delhi)'

In [14]:
neighborhood_latitude = df_delhi.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_delhi.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_delhi.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Ashok Nagar (Delhi) are 28.692230000000052, 77.30124000000006.


#### Pull the nearby venues

In [15]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&radius={}&limit={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, 1000, 100, VERSION)
results = requests.get(url).json()

In [16]:
results

{'meta': {'code': 200, 'requestId': '5fb400e8d91d7d1d49722662'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Delhi',
  'headerFullLocation': 'Delhi',
  'headerLocationGranularity': 'city',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 28.70123000900006,
    'lng': 77.31148064003828},
   'sw': {'lat': 28.683229991000044, 'lng': 77.29099935996184}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '56f0e1a5498e339a6f9cacd6',
       'name': 'Neeraj Kumar Garg',
       'location': {'address': 'Ashok Nagar',
        'crossStreet': 'New Ashok Nagar',
        'lat': 28.692731217431845,
        'lng': 77.29877207487533,
        'labeledLatLngs': [{'label': 'display',
         

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-18-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Neeraj Kumar Garg,Spa,28.692731,77.298772
1,Axis Bank ATM,ATM,28.696470,77.299910
2,Sutta Chowk,Smoke Shop,28.697897,77.300010
3,My Idea Store,Mobile Phone Shop,28.686405,77.299520
4,Axis Bank ATM,ATM,28.694360,77.308370


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
LIMIT = 100
delhi_venues = getNearbyVenues(names=df_delhi['Neighborhood'],
                                   latitudes=df_delhi['Latitude'],
                                   longitudes=df_delhi['Longitude']
                                  )

In [22]:
print(delhi_venues.shape)
delhi_venues.head(10)

(1163, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ashok Nagar (Delhi),28.69223,77.30124,Neeraj Kumar Garg,28.692731,77.298772,Spa
1,Ashok Nagar (Delhi),28.69223,77.30124,Axis Bank ATM,28.696470,77.299910,ATM
2,Ashok Vihar,28.69037,77.17609,Domino's Pizza,28.693000,77.177000,Pizza Place
3,Ashok Vihar,28.69037,77.17609,Sagar Ratna | सागर रतना,28.693381,77.177977,South Indian Restaurant
4,Ashok Vihar,28.69037,77.17609,Central park,28.690427,77.175620,Garden
5,Ashok Vihar,28.69037,77.17609,J Block Murga Market,28.687144,77.173035,Market
6,Babarpur,28.50738,77.30346,Axis Bank ATM,28.507370,77.303467,ATM
7,Babarpur,28.50738,77.30346,Axis Bank ATM,28.503840,77.302720,ATM
8,Babarpur,28.50738,77.30346,Axis Bank ATM,28.503830,77.302690,ATM
9,"Badarpur, Delhi",28.50738,77.30346,Axis Bank ATM,28.507370,77.303467,ATM


In [23]:
delhi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ashok Nagar (Delhi),2,2,2,2,2,2
Ashok Vihar,4,4,4,4,4,4
Babarpur,3,3,3,3,3,3
"Badarpur, Delhi",3,3,3,3,3,3
Bali Nagar,8,8,8,8,8,8
Ber Sarai,9,9,9,9,9,9
Chandni Chowk,14,14,14,14,14,14
"Civil Lines, Delhi",4,4,4,4,4,4
"Connaught Place, New Delhi",70,70,70,70,70,70


In [24]:
print('There are {} uniques categories.'.format(len(delhi_venues['Venue Category'].unique())))

There are 170 uniques categories.


#### One-hot encoding for the catogory of the venues

In [25]:
# one hot encoding
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighborhood'] = delhi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

delhi_onehot.head()

,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport Food Court,Airport Lounge,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Business Service,Café,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hindu Temple,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Internet Cafe,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Light Rail Station,Lighting Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mosque,Motel,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,Nightclub,North Indian Restaurant,Northeast Indian Restaurant,Nudist Beach,Office,Other Nightlife,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Record Shop,Resort,Rest Area,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Stadium,Steakhouse,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,University,Water Park,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [26]:
delhi_onehot.shape

(1163, 170)

In [27]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport Food Court,Airport Lounge,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Business Service,Café,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hindu Temple,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Internet Cafe,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Light Rail Station,Lighting Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mosque,Motel,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Nightclub,North Indian Restaurant,Northeast Indian Restaurant,Nudist Beach,Office,Other Nightlife,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Record Shop,Resort,Rest Area,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Stadium,Steakhouse,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,University,Water Park,Wine Bar,Women's Store
0,Ashok Nagar (Delhi),0.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000
1,Ashok Vihar,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000

In [28]:
delhi_grouped.shape

(139, 170)

In [29]:
fixed_columns

['Yoga Studio',
 'ATM',
 'Accessories Store',
 'Afghan Restaurant',
 'Airport Food Court',
 'Airport Lounge',
 'American Restaurant',
 'Arcade',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Australian Restaurant',
 'BBQ Joint',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Bed & Breakfast',
 'Beer Garden',
 'Bike Shop',
 'Bistro',
 'Bookstore',
 'Boutique',
 'Breakfast Spot',
 'Burger Joint',
 'Burmese Restaurant',
 'Bus Station',
 'Business Service',
 'Café',
 'Campground',
 'Chinese Restaurant',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'Comedy Club',
 'Comfort Food Restaurant',
 'Convenience Store',
 'Cosmetics Shop',
 'Cupcake Shop',
 'Deli / Bodega',
 'Department Store',
 'Dessert Shop',
 'Dhaba',
 'Diner',
 'Dog Run',
 'Donut Shop',
 'Eastern European Restaurant',
 'Electronics Store',
 'English Restaurant',
 'Event Space',
 'Fabric Shop',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Flea Market',
 'Flower

In [30]:
len(delhi_grouped[delhi_grouped["Bakery"] > 0])

20

#### Analyze the Bakery data

In [31]:
delhi_bak = delhi_grouped[["Neighborhood","Bakery"]]

In [32]:
delhi_bak.head(20)

,Neighborhood,Bakery
0,Ashok Nagar (Delhi),0.000000
1,Ashok Vihar,0.000000
2,Babarpur,0.000000
3,"Badarpur, Delhi",0.000000
4,Bali Nagar,0.000000
5,Ber Sarai,0.000000
6,Chandni Chowk,0.000000
7,"Civil Lines, Delhi",0.000000
8,"Connaught Place, New Delhi",0.014286
9,"Dabri, New Delhi",0.000000


### Clustering using K-Means clustering algorithm

In [33]:
# set number of clusters
kclusters = 5

delhi_clustering = delhi_bak.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [34]:
# create a new dataframe that includes the cluster labels
delhi_merged = delhi_bak.copy()

# add clustering labels
delhi_merged["Cluster Label"] = kmeans.labels_
delhi_merged.head()

,Neighborhood,Bakery,Cluster Label
0,Ashok Nagar (Delhi),0.0,1
1,Ashok Vihar,0.0,1
2,Babarpur,0.0,1
3,"Badarpur, Delhi",0.0,1
4,Bali Nagar,0.0,1


In [35]:
# merge delhi_grouped with delhi_data to add latitude/longitude for each neighborhood
delhi_merged = delhi_merged.join(df_delhi.set_index("Neighborhood"), on="Neighborhood")

print(delhi_merged.shape)
delhi_merged.head() # check the last columns!

(139, 5)


,Neighborhood,Bakery,Cluster Label,Latitude,Longitude
0,Ashok Nagar (Delhi),0.0,1,28.69223,77.301240
1,Ashok Vihar,0.0,1,28.69037,77.176090
2,Babarpur,0.0,1,28.50738,77.303460
3,"Badarpur, Delhi",0.0,1,28.50738,77.303460
4,Bali Nagar,0.0,1,28.65223,77.129411


In [36]:
# Sort by cluster labels
delhi_merged.sort_values(["Cluster Label"], inplace=True)
delhi_merged

,Neighborhood,Bakery,Cluster Label,Latitude,Longitude
72,Munirka,0.066667,0,28.555040,77.171320
30,Greater Kailash,0.052632,0,28.548490,77.236670
45,Karol Bagh,0.062500,0,28.650450,77.188730
42,Kailash Colony,0.058824,0,28.556130,77.240600
0,Ashok Nagar (Delhi),0.000000,1,28.692230,77.301240
90,Pamposh Enclave,0.000000,1,28.544430,77.245650
91,Pandav Nagar,0.000000,1,28.614580,77.275740
92,Paschim Vihar,0.000000,1,28.669330,77.091730
93,Patel Nagar,0.000000,1,28.647830,77.164490
94,Pitam Pura,0.000000,1,28.695900,77.137250


#### Map Visualization - Neighborhood of  New Delhi Cluster wise

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(delhi_merged['Latitude'], delhi_merged['Longitude'], delhi_merged['Neighborhood'], delhi_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:

# save the map as HTML file
map_clusters.save('map_clusters.html')

### Analyze the Clusters
#### First Cluster (Cluster Label - 0)

In [39]:
delhi_merged.loc[delhi_merged['Cluster Label'] == 0]

,Neighborhood,Bakery,Cluster Label,Latitude,Longitude
72,Munirka,0.066667,0,28.55504,77.17132
30,Greater Kailash,0.052632,0,28.54849,77.23667
45,Karol Bagh,0.062500,0,28.65045,77.18873
42,Kailash Colony,0.058824,0,28.55613,77.24060


#### Second Cluster (Cluster Label - 1)

In [40]:
delhi_merged.loc[delhi_merged['Cluster Label'] == 1]

,Neighborhood,Bakery,Cluster Label,Latitude,Longitude
0,Ashok Nagar (Delhi),0.000000,1,28.692230,77.301240
90,Pamposh Enclave,0.000000,1,28.544430,77.245650
91,Pandav Nagar,0.000000,1,28.614580,77.275740
92,Paschim Vihar,0.000000,1,28.669330,77.091730
93,Patel Nagar,0.000000,1,28.647830,77.164490
94,Pitam Pura,0.000000,1,28.695900,77.137250
95,Preet Vihar,0.000000,1,28.639030,77.295970
96,Punjabi Bagh,0.000000,1,28.666340,77.125000
97,Raisina Hill,0.000000,1,28.618400,77.215481
98,"Rajendra Nagar, Delhi",0.000000,1,28.590750,77.227490


#### Third Cluster (Cluster Label - 2)

In [41]:

delhi_merged.loc[delhi_merged['Cluster Label'] == 2]

,Neighborhood,Bakery,Cluster Label,Latitude,Longitude
17,Derawal Nagar,0.200000,2,28.699110,77.19105
39,Janakpuri,0.200000,2,28.627910,77.09060
18,Dhaula Kuan,0.166667,2,28.592378,77.15948
134,Vasundhara Enclave,0.250000,2,28.600150,77.31663


#### Fourth Cluster (Cluster Label - 3)

In [42]:
delhi_merged.loc[delhi_merged['Cluster Label'] == 3]

,Neighborhood,Bakery,Cluster Label,Latitude,Longitude
71,Mukherjee Nagar,0.333333,3,28.71053,77.21440
52,"Krishna Nagar, Delhi",0.333333,3,28.65545,77.28336


#### Fifth Cluster (Cluster Label - 4)

In [43]:

delhi_merged.loc[delhi_merged['Cluster Label'] == 4]

,Neighborhood,Bakery,Cluster Label,Latitude,Longitude
27,Gole Market,0.090909,4,28.634100,77.20569
15,Defence Colony,0.086957,4,28.572980,77.23357
100,Rajouri Garden,0.095238,4,28.645620,77.12209
62,Malviya Nagar (Delhi),0.090909,4,28.533940,77.20702
64,Mayur Vihar,0.090909,4,28.607714,77.29067
133,"Vasant Vihar, Delhi",0.125000,4,28.564940,77.16131


### Result and Discussion


After careful analysis of all five clusters, It's clear that the Places which are part of Second cluster (Cluster Label - 1) are most suitable for opening a new Bakery.

Second cluster (Cluster Label - 1) has least no of existing Bakery and at the same time places in Cluster Label-1 are well connected to Airport/Railway Station/Bus Station and other popular public places.
Cluster Label-1 suggests better business locations to open new Bakery.

##### So, This analysis suggests to open a new Bakery in the Second cluster (Cluster Label - 1)

## Conclusion

The purpose of this project was to explore the city of New Delhi and to find the best suitable places to open a new Bakery. We explored the city based on their neighbourhood and there famous venues present in each of the neighbourhoods finally concluding with clustering similar neighbourhoods together.

After careful analysis of all five clusters, It's clear that the Places which are part of Second cluster (Cluster Label - 1) are most suitable for opening a new Bakery. Second cluster has the least no of existing Bakery but It has very good connectivity to other popular public places.
This analysis suggests to open a new Bakery in the location mentioned in Second cluster (Cluster Label - 1).

